In [1]:
%cd "OneDrive - National University of Singapore"

C:\Users\wongp\OneDrive - National University of Singapore


In [2]:
%cd CS4248/Project/db/

C:\Users\wongp\OneDrive - National University of Singapore\CS4248\Project\db


In [3]:
import os
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import Counter
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

Convert training data into csv file

In [ ]:
import os
import pandas as pd

pathNeg = "train/neg"
files = os.listdir(pathNeg)
data = []
for file in files:
  if os.path.isfile(os.path.join(pathNeg, file)):
    id = int(file.split("_")[0])
    with open(os.path.join(pathNeg, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 0])

sizeOfNeg = 12500

pathPos = "train/pos"
files = os.listdir(pathPos)
for file in files:
  if os.path.isfile(os.path.join(pathPos, file)):
    id = int(file.split("_")[0]) + sizeOfNeg
    with open(os.path.join(pathPos, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 1])


df = pd.DataFrame(data, columns = ['id', 'text', 'score']).set_index('id').sort_index()
df.to_csv('train.csv')

Convert testing data into csv file

In [ ]:
pathNeg = "test/neg"
files = os.listdir(pathNeg)
data = []
for file in files:
  if os.path.isfile(os.path.join(pathNeg, file)):
    id = int(file.split("_")[0])
    with open(os.path.join(pathNeg, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 0])

sizeOfNeg = 12500

pathPos = "test/pos"
files = os.listdir(pathPos)
for file in files:
  if os.path.isfile(os.path.join(pathPos, file)):
    id = int(file.split("_")[0]) + sizeOfNeg
    with open(os.path.join(pathPos, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 1])


df = pd.DataFrame(data, columns = ['id', 'text', 'score']).set_index('id').sort_index()
df.to_csv('test.csv')

convert vocab into csv file

In [ ]:
with open("imdb.vocab", encoding='utf8') as f:
  vocab = []
  for line in f:
    vocab.append(line.rstrip())

vocab_df = pd.DataFrame(vocab, columns=['vocab'])
print(vocab_df)
vocab_df.to_csv('vocab_given.csv', index=False)

                   vocab
0                    the
1                    and
2                      a
3                     of
4                     to
...                  ...
89522          copywrite
89523             artbox
89524          kinky-sex
89525           urrrghhh
89526  investigator-like

[89527 rows x 1 columns]


In [4]:
from nltk.tokenize import word_tokenize

def tokenizeReview(review):
  review = review.replace('.', ' . ')
  tokens = word_tokenize(review.lower())
  return tokens

Get vocab from training set

In [ ]:
from collections import Counter
reviews = train_df['text'].tolist()
reviews = list(map(tokenizeReview, reviews))
reviews = [item for review in reviews for item in review]
vdict = Counter(reviews).most_common()

In [ ]:
vocabs = []
for pair in vdict:
  vocabs.append(pair[0])

vocab_df = pd.DataFrame(vocabs, columns=['vocab'])
vocab_df.to_csv('vocab.csv', index=False)
print(vocab_df)

              vocab
0               the
1                 .
2                 ,
3               and
4                 a
...             ...
99203    people-and
99204         padre
99205   unmatchable
99206  scenery-wise
99207  non-teenager

[99208 rows x 1 columns]


Read training data and testing data from csv file (START HERE after initialization)

In [5]:
train_df = pd.read_csv('train/train.csv')
test_df = pd.read_csv('test/test.csv')
vocab_df = pd.read_csv('vocab.csv')
print(vocab_df)

              vocab
0               the
1                 .
2                 ,
3               and
4                 a
...             ...
99203    people-and
99204         padre
99205   unmatchable
99206  scenery-wise
99207  non-teenager

[99208 rows x 1 columns]


In [ ]:
vocab_dict = pd.Series(vocab_df.index + 1, index=vocab_df.vocab).to_dict()
print(vocab_dict)

In [7]:
def convertToIndex(review):
  vec = []
  for word in review:
    if word in vocab_dict:
      vec.append(int(vocab_dict[word]))
    else:
      vec.append(0)
  return vec

In [8]:
train_df['tokens'] = train_df['text'].apply(tokenizeReview)

In [9]:
train_df['vec'] = train_df['tokens'].apply(convertToIndex)

In [10]:
train_df

,id,text,score,tokens,vec
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli...","[76, 6, 5, 142, 46, 56, 7518, 1387, 21, 5, 451..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl...","[649, 5006, 307, 1, 106, 3769, 1076, 9203, 6, ..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea...","[15, 226, 1, 9586, 906, 311, 28, 102, 129, 2, ..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio...","[57, 15, 78, 32, 338, 3189, 7720, 18, 34296, 8..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ...","[16, 16016, 213, 512, 56, 3154, 1789, 397, 5, ..."
...,...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, :, some, inventive...","[60, 18, 62, 55, 13, 87, 61, 4388, 4, 2028, 93..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h...","[21, 168, 82, 155, 93, 163, 188, 213, 25, 17, ..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi...","[57, 29, 37, 31, 466, 126, 16, 23, 6, 13586, 4..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ...","[16, 23, 8, 1, 133, 23, 138, 13, 56, 5, 186, 6..."


In [ ]:
print('Maximum review length: {}'.format(
len(max((train_df['tokens']), key=len))))
print('Minimum review length: {}'.format(
len(min((train_df['tokens']), key=len))))

Maximum review length: 2820
Minimum review length: 11


In [11]:
max_words = 500
dataset = sequence.pad_sequences(train_df['vec'], maxlen=max_words)
y = train_df['score']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(dataset, y, test_size=0.3, stratify=y, random_state = 42)

In [12]:
vocab_size = len(vocab_dict) + 1
print(vocab_size)

99208


In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU
embedding_size=32
model=Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_words))
model.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           3174656   
                                                                 
 gru_2 (GRU)                 (None, 500, 50)           12600     
                                                                 
 dense_1 (Dense)             (None, 500, 1)            51        
                                                                 
Total params: 3,187,307
Trainable params: 3,187,307
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
547/547 [==============================] - 75s 135ms/step - loss: 0.6234 - accuracy: 0.6098
Epoch 2/5
547/547 [==============================] - 73s 133ms/step - loss: 0.5100 - accuracy: 0.6797
Epoch 3/5
547/547 [==============================] - 75s 137ms/step - loss: 0.4461 - accuracy: 0.7077
Epoch 4/5
547/547 [==============================] - 74s 135ms/step - loss: 0.4130 - accuracy: 0.7252
Epoch 5/5
547/547 [==============================] - 74s 135ms/step - loss: 0.3973 - accuracy: 0.7269


In [ ]:
scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.6569069623947144


In [ ]:
model.save('basic_model')

INFO:tensorflow:Assets written to: basic_gru_model\assets


INFO:tensorflow:Assets written to: basic_gru_model\assets


**Without 'return-sequences=True' parameter**

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU
embedding_size=32
model=Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_words))
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 500, 32)           3174656   
                                                                 
 gru_9 (GRU)                 (None, 50)                12600     
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 3,187,307
Trainable params: 3,187,307
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
547/547 [==============================] - 75s 134ms/step - loss: 0.5092 - accuracy: 0.7413
Epoch 2/5
547/547 [==============================] - 73s 134ms/step - loss: 0.2315 - accuracy: 0.9130
Epoch 3/5
547/547 [==============================] - 73s 134ms/step - loss: 0.1140 - accuracy: 0.9619
Epoch 4/5
547/547 [==============================] - 73s 133ms/step - loss: 0.0594 - accuracy: 0.9807
Epoch 5/5
547/547 [==============================] - 72s 132ms/step - loss: 0.0554 - accuracy: 0.9798


In [ ]:
scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.8438666462898254


In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU
embedding_size=32
model=Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_words))
#model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
model.add(GRU(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 500, 32)           3174656   
                                                                 
 gru_13 (GRU)                (None, 100)               40200     
                                                                 
 dense_12 (Dense)            (None, 1)                 101       
                                                                 
Total params: 3,214,957
Trainable params: 3,214,957
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
547/547 [==============================] - 94s 170ms/step - loss: 0.5660 - accuracy: 0.7101
Epoch 2/5
547/547 [==============================] - 92s 168ms/step - loss: 0.2973 - accuracy: 0.8813
Epoch 3/5
547/547 [==============================] - 92s 168ms/step - loss: 0.1586 - accuracy: 0.9

# Using Test Dataest

In [23]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_size=32
model=Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_words))
model.add(GRU(units=50, return_sequences = True, input_shape=(dataset.shape[1],1), activation='tanh'))
#model.add(GRU(100))
model.add(Dropout(0.6))
model.add(GRU(units=50, input_shape=(dataset.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 500, 32)           3174656   
                                                                 
 gru_16 (GRU)                (None, 500, 50)           12600     
                                                                 
 dropout_7 (Dropout)         (None, 500, 50)           0         
                                                                 
 gru_17 (GRU)                (None, 50)                15300     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 3,202,607
Trainable params: 3,202,607
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
batch_size=32
model.fit(dataset, y, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
782/782 [==============================] - 201s 254ms/step - loss: 0.4726 - accuracy: 0.7640
Epoch 2/5
782/782 [==============================] - 196s 250ms/step - loss: 0.2174 - accuracy: 0.9174
Epoch 3/5
782/782 [==============================] - 197s 252ms/step - loss: 0.1112 - accuracy: 0.9625
Epoch 4/5
782/782 [==============================] - 198s 254ms/step - loss: 0.0584 - accuracy: 0.9820
Epoch 5/5
782/782 [==============================] - 198s 253ms/step - loss: 0.0324 - accuracy: 0.9900


In [25]:
test_df['tokens'] = test_df['text'].apply(tokenizeReview)
test_df['vec'] = test_df['tokens'].apply(convertToIndex)
test_x = sequence.pad_sequences(test_df['vec'], maxlen=max_words)
test_x

array([[    0,     0,     0, ...,   569,    14,     2],
       [    0,     0,     0, ...,    18,   300,     2],
       [    0,     0,     0, ..., 12518,    83,     2],
       ...,
       [    0,     0,     0, ...,   768,  2526,     2],
       [    0,     0,     0, ...,    42,   220,     2],
       [    0,     0,     0, ...,   899,  2388,     2]])

In [26]:
test_y = test_df['score']
scores = model.evaluate(test_x, test_y, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.8578799962997437


In [27]:
pred = model.predict(test_x)

from sklearn.metrics import accuracy_score
pred_labels = []
for i in pred:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_y,pred_labels))



Accuracy of prediction on test set :  0.85788


In [28]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(test_y,pred_labels)
print(cm)

[[10645  1855]
 [ 1698 10802]]


In [29]:
test_df['pred'] = pred_labels
test_df
test_df.to_csv("test_df_pred_base.csv")